In [1]:
import os
import sys
import csv
import requests
import json
import operator
import time
import datetime
import platform
import itertools

In [2]:
def writeText(text, path, mode = 'a'):
    with open (path, mode) as textout:
        textout.write((text))
        
def writeJson(json, path, mode = 'w'):
    with open(path, mode) as file:
        file.write(json.dumps(json))
        
def writeCsv(listOut, outputFile):
    import csv
    with open (outputFile, "w", newline='', encoding = 'utf-8') as outputfile:
        writer = csv.writer(outputfile, delimiter = ",")
        for element in listOut:
            writer.writerow(element)

In [3]:
def getTxt(path):
    return open(path, 'r').read()

def getCsv(path, delim = ','):
    list_return = []
    with open (path, encoding = 'windows-1252') as file:
        csvreader = csv.reader(file, delimiter = delim)        
        for i, line in enumerate(csvreader):
            list_inner = []
            list_inner.append(line[0].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_inner.append(line[1].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_inner.append(line[2].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_return.append(list_inner)
    
    
    return list_return

In [4]:
if platform.system() == 'Windows':
    prep_data = getCsv('..\\output\\prep_out.csv')
else:
    prep_data = getCsv('../output/prep_out.csv')

In [5]:
print(prep_data[1])

[['domestic', ' general', ' government', ' health', ' expenditure', ' capita', ' current', ' us', ' $', ' world', ' regions'], ['public', ' expenditure', ' health', ' domestic', ' sources', ' capita', ' expressed', ' current', ' us', ' dollars'], ['social', ' issues', ' environment', ' politics', ' national', ' accounts', ' health', ' systems', ' by', ' country', ' military', ' budget']]


## Word2Vec

In [6]:
def getVecList(list_in):
    failcnt = 0
    str_log = ''
    print('Started at:')
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'),'\n')
    # List of Lists as input needed
#     adress = 'http://word2vec.ai.wu.ac.at/googlenews/model?word='
    adress = 'http://word2vec.ai.wu.ac.at/twitter27_200/model?word='
    list_return = [[],[],[]]
    list_failed = []
    
    for i, elem in enumerate(list_in):
        inner_feat1, inner_feat2, inner_label = [], [], []
        str_1 = '-------------------------------\n' + 'Element ' + str(i)
        #print(str_1)
        str_log += str_1 + '\n'
# name ----------------------------------------
        for str_elem in elem[0]:
            word = str_elem.strip() 
            try:
                inner_feat1.append([float(x) for x in requests.get((adress+word)).text.replace(' ','').replace('[','').replace(']','').split(',')])
            except:
                try:
                    inner_feat1.append([float(x) for x in requests.get((adress+word.title())).text.replace(' ','').replace('[','').replace(']','').split(',')])
                except Exception as e:
                    str_2 = 'fail\n' + adress+(word) + '\n' + str(e) + '\n' + requests.get((adress+word)).text
                    #print(str_2)
                    str_log += str_2 + '\n'
                    inner_feat1.append([])
                    list_failed.append(word)
                    failcnt += 1
# desc ----------------------------------------
        for str_elem in elem[1]:
            word = str_elem.strip() 
            try:
                inner_feat2.append([float(x) for x in requests.get((adress+word)).text.replace(' ','').replace('[','').replace(']','').split(',')])
            except:
                try:
                    inner_feat2.append([float(x) for x in requests.get((adress+word.title())).text.replace(' ','').replace('[','').replace(']','').split(',')])
                except Exception as e:
                    str_3 = 'fail\n' + adress+(word) + '\n' + str(e) + '\n' + requests.get((adress+word)).text
                    #print(str_3)
                    str_log += str_3 + '\n'
                    inner_feat2.append([])
                    list_failed.append(word)
                    failcnt += 1
# label ----------------------------------------
        for str_elem in elem[2]:
            word = str_elem.strip() 
            try:
                inner_label.append([float(x) for x in requests.get((adress+word)).text.replace(' ','').replace('[','').replace(']','').split(',')])
            except:
                try:
                    inner_label.append([float(x) for x in requests.get((adress+word.title())).text.replace(' ','').replace('[','').replace(']','').split(',')])
                except Exception as e:
                    str_4 = 'fail\n' + adress+(word) + '\n' + str(e) + '\n' + requests.get((adress+word)).text
                    #print(str_4)
                    str_log += str_4 + '\n'
                    inner_label.append([])
                    list_failed.append(word)
                    failcnt += 1
        
        
        list_return[0].append(inner_feat1)
        list_return[1].append(inner_feat2)
        list_return[2].append(inner_label)
        
        
    print('Failed words: ', failcnt)
    writeText(str_log, '../output/log.txt')    
    print('\nEnded at:')
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')) 
    return list_return, list_failed

In [7]:
vect_list = getVecList(prep_data)

Started at:
2018-10-31 19:06:01 

Failed words:  5163

Ended at:
2018-10-31 19:23:50


In [8]:
feature1 = vect_list[0][0]
feature2 = vect_list[0][1]
labels = vect_list[0][2]
missing = vect_list[1]

In [13]:
print(feature1[0])
print('-----------------------------------')
print(feature2[0])
print('-----------------------------------')
print(labels[0])

[[-0.8230699896812439, -0.2092200070619583, -0.37755000591278076, 0.4268699884414673, 0.02107900008559227, -0.28810998797416687, 0.5888400077819824, 0.010553999803960323, 0.22157999873161316, -0.7311199903488159, -0.19881999492645264, 0.4277699887752533, -1.1519999504089355, 0.20905999839305878, 0.6625800132751465, -0.0890209972858429, -0.07255499809980392, 0.26579999923706055, -0.13710999488830566, -0.49015000462532043, -0.1811400055885315, -0.05023900046944618, -0.5853999853134155, 0.3596799969673157, -0.7145699858665466, 1.2523000240325928, 0.05327799916267395, 0.46542999148368835, -0.48824000358581543, 0.5087000131607056, 0.15068000555038452, -0.2312999963760376, -0.8912500143051147, 0.06969200074672699, -0.30107998847961426, 0.06497500091791153, -0.3909200131893158, 0.5113400220870972, -0.0058200000785291195, -0.3213900029659271, 0.15563000738620758, 0.06980500370264053, 0.8106399774551392, -0.6601600050926208, -0.040911998599767685, -0.5463700294494629, -0.006608800031244755, -1.

In [10]:
if platform.system() == 'Windows':
    writeCsv(feature1, '..\\output\\feat1.csv')
    writeCsv(feature1, '..\\output\\feat2.csv')
    writeCsv(feature1, '..\\output\\labels.csv')
else:
    writeCsv(feature1, '../output/feat1.csv')
    writeCsv(feature1, '../output/feat2.csv')
    writeCsv(feature1, '../output/labels.csv')